### CTR : 전체 페이지 뷰 횟수 대비 특정 광고를 클릭한 횟수의 비윤

In [19]:
import pandas as pd

In [20]:
train_df = pd.read_csv('./train.csv', nrows = 100000)     # 학습 셋의 행을 100000개로 제한하여 데이터프레임 생성
train_df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [21]:
unused_columns, label_column = ['id','hour','device_id','device_ip'], 'click'
# 필요없는 컬럼과 레이블 컬럼 리스트 형태로

train_df = train_df.drop(unused_columns, axis = 1) 
# 필요없는 컬럼을 열을 기준으로 없애라

X_dict_train = list(train_df.drop(label_column, axis = 1).T.to_dict().values())
# 레이블 컬럼을 제외한 나머지 데이터를 dic 형태의 리스트로 변환하라

X_dict_train

[{'C1': 1005,
  'banner_pos': 0,
  'site_id': '1fbe01fe',
  'site_domain': 'f3845767',
  'site_category': '28905ebd',
  'app_id': 'ecad2386',
  'app_domain': '7801e8d9',
  'app_category': '07d7df22',
  'device_model': '44956a24',
  'device_type': 1,
  'device_conn_type': 2,
  'C14': 15706,
  'C15': 320,
  'C16': 50,
  'C17': 1722,
  'C18': 0,
  'C19': 35,
  'C20': -1,
  'C21': 79},
 {'C1': 1005,
  'banner_pos': 0,
  'site_id': '1fbe01fe',
  'site_domain': 'f3845767',
  'site_category': '28905ebd',
  'app_id': 'ecad2386',
  'app_domain': '7801e8d9',
  'app_category': '07d7df22',
  'device_model': '711ee120',
  'device_type': 1,
  'device_conn_type': 0,
  'C14': 15704,
  'C15': 320,
  'C16': 50,
  'C17': 1722,
  'C18': 0,
  'C19': 35,
  'C20': 100084,
  'C21': 79},
 {'C1': 1005,
  'banner_pos': 0,
  'site_id': '1fbe01fe',
  'site_domain': 'f3845767',
  'site_category': '28905ebd',
  'app_id': 'ecad2386',
  'app_domain': '7801e8d9',
  'app_category': '07d7df22',
  'device_model': '8a4875b

In [23]:
y_train = train_df[label_column]

test_df = pd.read_csv('train.csv', header=0, skiprows=(1,100000), nrows=100000)
# index=0은 컬럼이름으로 사용, index=1 건너뛰고, nrows는 파일에서 첨부터 최대 100,000개의 행만 읽어옴
test_df = test_df.drop(unused_columns, axis=1)

X_dict_test = list(test_df.drop(label_column, axis=1).T.to_dict().values())
# 모델이 예측 가능한 데이터 준비. 
y_test = test_df[label_column]

In [26]:
from sklearn.feature_extraction import DictVectorizer    # dic형태의 데이터를 ML모델이 이해할 수 있는 숫자 형태의 벡터로 변환

In [32]:
vectorizer = DictVectorizer(sparse =True)     
# 변환된 결과물을 희소 행렬 형태로 만들라는 파라미터 < 값이 0인 행렬에서 메모리 사용 효율성을 높이기 위해 사용

X_train = vectorizer.fit_transform(X_dict_train)   
# 객체 생성 후 X_dict_train 데이터의 각 딕셔너리를 벡터로 변환
X_train.shape

(100000, 4952)

In [35]:
X_train.toarray()[:5]  # 희소 행렬을 밀집 배열로 변환하여 인덱스 5이전까지 행 선택 ( np 배열 형태)

array([[ 1005., 15706.,   320., ...,     0.,     0.,     0.],
       [ 1005., 15704.,   320., ...,     0.,     0.,     0.],
       [ 1005., 15704.,   320., ...,     0.,     0.,     0.],
       [ 1005., 15706.,   320., ...,     0.,     0.,     0.],
       [ 1005., 18993.,   320., ...,     0.,     0.,     0.]])

In [37]:
X_test = vectorizer.fit_transform(X_dict_test)

X_test

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1900000 stored elements and shape (100000, 4952)>

In [38]:
from sklearn.tree import DecisionTreeClassifier

parameters = {"max_depth" : [3, 10, None]}

decision_tree = DecisionTreeClassifier(criterion = 'gini', min_samples_split=30)

In [39]:
from sklearn.model_selection import GridSearchCV

In [41]:
grid_search = GridSearchCV(decision_tree, parameters, n_jobs= -1, cv=3 , scoring='roc_auc')

In [42]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(min_samples_split=30),
             n_jobs=-1, param_grid={'max_depth': [3, 10, None]},
             scoring='roc_auc')

In [44]:
grid_search.best_params_

{'max_depth': 10}

In [45]:
decision_tree_best = grid_search.best_estimator_

>의사결정 트리모델의 성능 측정, 정확도, 혼동행렬, ROC의 AUC

- 실제 관측값을 얼마나 정확하게 예측했는지를 보여주는 행렬
- TP 참 - 긍정
- TN 참 - 부정
- FP 거짓 - 긍정 (1형 오류)
- FN 거짓 - 부정 (2형 오류)

In [47]:
y_pred= decision_tree_best.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [49]:
import numpy as np

In [50]:
np.unique(y_pred, return_counts= True)

(array([0, 1]), array([96545,  3455]))

In [51]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)     # 정확도 

0.83249

In [53]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)    # 혼동행렬

array([[81152,  1358],
       [15393,  2097]])